### Data Preparation
Here we import the required libraries and load our datasets. We use `pandas` for data manipulation, `numpy` for numerical operations, and `matplotlib` for plotting. Additionally, we use scikit-learn for encoding categorical variables and splitting the data, and TensorFlow with Keras for building the neural network model. After loading, we pre-process the data by selecting the relevant columns and encoding the user and business IDs.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Input, Dot, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

pd.set_option('display.max_columns', None)

In [2]:
review = pd.read_csv('review.csv')
business = pd.read_csv('business.csv')


In [3]:
review = review[['user_id','business_id','stars']]

In [4]:
train_data, test_data = train_test_split(review, test_size=0.2)

### Encoding User and Business IDs
To handle categorical data in our model, we encode the user and business IDs using label encoding, which converts each unique string into a numerical representation. This is a necessary step before feeding the data into our neural network model. The encoding process is applied to both the training and testing datasets to ensure consistency.


In [5]:

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

train_data['user_id_encoded'] = user_encoder.fit_transform(train_data['user_id'])
train_data['business_id_encoded'] = business_encoder.fit_transform(train_data['business_id'])

In [6]:
test_data = test_data[test_data['user_id'].isin(user_encoder.classes_)]
test_data = test_data[test_data['business_id'].isin(business_encoder.classes_)]

In [7]:
test_data['user_id_encoded'] = user_encoder.transform(test_data['user_id'])
test_data['business_id_encoded'] = business_encoder.transform(test_data['business_id'])

In [8]:
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

print(f"Unique Users: {num_users}, Unique Businesses: {num_businesses}")

Unique Users: 75510, Unique Businesses: 1705


### Defining the Neural Network Model
Here we define our neural network model architecture. We use embeddings to capture the latent factors of users and businesses and concatenate these embeddings to form the input to the dense layers of the network. The model aims to predict user ratings for businesses. We use a mean squared error loss function and the Adam optimizer.


In [9]:
embedding_dim=32

user_input = Input(shape=(1,), name='user_input')
business_input = Input(shape=(1,), name='business_input')

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
business_embedding = Embedding(input_dim=num_businesses, output_dim=embedding_dim, embeddings_regularizer=l2(1e-6))(business_input)

user_flatten = Flatten()(user_embedding)
business_flatten = Flatten()(business_embedding)

merged = Concatenate()([user_flatten, business_flatten])
merged = BatchNormalization()(merged)

dense_layer = Dense(128, activation='relu')(merged)
dropout = Dropout(0.4)(dense_layer)
output_layer = Dense(1, activation='linear')(dropout)

model = Model(inputs=[user_input, business_input], outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

### Model Training
In this section, we fit the model to our training data using a batch size of 128 and a specified number of epochs. We implement early stopping to prevent overfitting and a model checkpoint to save the best weights during training. We monitor the validation loss and stop training if it doesn't improve after a defined number of epochs.


In [10]:
batch_size = 128
epochs = 20

user_ids = train_data['user_id_encoded'].values
business_ids = train_data['business_id_encoded'].values
stars = train_data['stars'].values


In [11]:
model_checkpoint = ModelCheckpoint(f'./model/model.weights.h5',
                             monitor='val_loss',   # Monitor validation loss
                             save_best_only=True,  # Save only the best model
                             save_weights_only=True,
                             mode='min'            # Mode of monitoring (minimize validation loss)
                            )

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True
                              )

In [12]:
history = model.fit(
    [user_ids, business_ids],
    stars,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 3.7999 - mae: 1.5459 - val_loss: 1.7116 - val_mae: 1.0588
Epoch 2/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.5151 - mae: 0.9889 - val_loss: 1.7550 - val_mae: 1.0786
Epoch 3/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.0752 - mae: 0.8187 - val_loss: 1.8031 - val_mae: 1.0868
Epoch 4/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.8608 - mae: 0.7226 - val_loss: 1.8230 - val_mae: 1.0957
Epoch 5/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.7522 - mae: 0.6709 - val_loss: 1.8052 - val_mae: 1.0844
Epoch 6/20
1222/1222 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6619 - mae: 0.6270 - val_loss: 1.8151 - val_mae: 1.0887


### Model Evaluation
Once training is complete, we use our test dataset to evaluate the model's performance. We predict the ratings and calculate the mean squared error between the predicted and actual ratings to understand the model's accuracy.


In [13]:
test_user_ids = test_data['user_id_encoded'].values
test_business_ids = test_data['business_id_encoded'].values
test_stars = test_data['stars'].values



In [14]:
predictions = model.predict([test_user_ids, test_business_ids])

1146/1146 ━━━━━━━━━━━━━━━━━━━━ 1s 404us/step


In [15]:
mean_squared_error(predictions,test_stars)

1.573513826411443

In [16]:
%%time
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

review['user_id_encoded'] = user_encoder.fit_transform(review['user_id'])
review['business_id_encoded'] = business_encoder.fit_transform(review['business_id'])

CPU times: user 166 ms, sys: 4.9 ms, total: 171 ms
Wall time: 170 ms


In [17]:
num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

print(f"Unique Users: {num_users}, Unique Businesses: {num_businesses}")

Unique Users: 87090, Unique Businesses: 1705


In [18]:
embedding_dim=32

user_input = Input(shape=(1,), name='user_input')
business_input = Input(shape=(1,), name='business_input')

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
business_embedding = Embedding(input_dim=num_businesses, output_dim=embedding_dim, embeddings_regularizer=l2(1e-6))(business_input)

user_flatten = Flatten()(user_embedding)
business_flatten = Flatten()(business_embedding)

merged = Concatenate()([user_flatten, business_flatten])
merged = BatchNormalization()(merged)

dense_layer = Dense(128, activation='relu')(merged)
dropout = Dropout(0.4)(dense_layer)
output_layer = Dense(1, activation='linear')(dropout)

model = Model(inputs=[user_input, business_input], outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [19]:
batch_size = 128
epochs = 10

user_ids = review['user_id_encoded'].values
business_ids = review['business_id_encoded'].values
stars = review['stars'].values


### Saving Encoders and Model
After training and evaluation, we save the label encoders and the trained model to disk. This allows us to reload the trained model and encoders for future predictions without retraining from scratch.


In [20]:
model_checkpoint = ModelCheckpoint(f'./model/model.weights.h5',
                             monitor='val_loss',   # Monitor validation loss
                             save_best_only=True,  # Save only the best model
                             save_weights_only=True,
                             mode='min'            # Mode of monitoring (minimize validation loss)
                            )

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=1,
                               restore_best_weights=True
                              )

In [21]:
history = model.fit(
    [user_ids, business_ids],
    stars,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10


1528/1528 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - loss: 3.1757 - mae: 1.4261 - val_loss: 2.7720 - val_mae: 1.4321
Epoch 2/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 1.4790 - mae: 0.9775 - val_loss: 2.4720 - val_mae: 1.3386
Epoch 3/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 1.0670 - mae: 0.8126 - val_loss: 2.4136 - val_mae: 1.3080
Epoch 4/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - loss: 0.8661 - mae: 0.7221 - val_loss: 2.4012 - val_mae: 1.2962
Epoch 5/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.7475 - mae: 0.6651 - val_loss: 2.3900 - val_mae: 1.3017
Epoch 6/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.6778 - mae: 0.6305 - val_loss: 2.3493 - val_mae: 1.2849
Epoch 7/10
1528/1528 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.6250 - mae: 0.6025 - val_loss: 2.4385 - val_mae: 1.3275


In [22]:
import pickle

with open('./model/user_encoder.pickle', 'wb') as f:
    pickle.dump(user_encoder, f)
    
with open('./model/business_encoder.pickle', 'wb') as f:
    pickle.dump(business_encoder, f)